# Récupération des identifiants où il y a des animaux dans les images

Pour ce faire, nous allons récupérer la liste des différents animaux existant sur Terre grace à WikiData avec une request SPARQL :

```sparql
SELECT DISTINCT ?animal ?animalLabel WHERE {
  ?animal wdt:P31 wd:Q16521.
  ?animal rdfs:label ?animalLabel filter (lang(?animalLabel) = "en").
  #OPTIONAL {?animal wdt:P18 ?image.}
} LIMIT 1000000
```

Nous téléchargeons le fichier CSV pour les 1000000 premiers résultats et nous le sauvegardons dans le fichier `animals.csv`.

Sur Kaggle (https://www.kaggle.com/competitions/imagenet-object-localization-challenge/data?select=LOC_synset_mapping.txt), nous récupérons le fichier `LOC_synset_mapping.txt`, qui contient la liste des identifiants des animaux avec leur nom.

La correspondance entre les 1000 synset id et leurs descriptions. Par exemple, la ligne 1 indique n01440764 tanche, Tinca tinca signifie qu'il s'agit de la classe 1, que l'identifiant synset est n01440764 et qu'elle contient le poisson tanche.

```txt
1	n01440764 tench, Tinca tinca
2	n01443537 goldfish, Carassius auratus
```

Commençons par charger les données dans un DataFrame pandas. Nous allons utiliser le fichier `LOC_synset_mapping.txt` pour récupérer les identifiants des animaux et leur nom.

In [4]:
import pandas as pd

# Lecture du fichier texte
with open('LOC_synset_mapping.txt', 'r') as f:
    lignes = f.readlines()

# Création d'un DataFrame
df = pd.DataFrame({'ligne': lignes})

# Séparation de l'ID et du nom
df['id'] = df['ligne'].str.split(' ', n=1, expand=True)[0]
df['name'] = df['ligne'].str.split(' ', n=1, expand=True)[1].str.rstrip().str.split(',')

df.drop(columns=['ligne'], inplace=True)
# Affichage du DataFrame
df.head()

,id,name
0,n01440764,"[tench, Tinca tinca]"
1,n01443537,"[goldfish, Carassius auratus]"
2,n01484850,"[great white shark, white shark, man-eater, ..."
3,n01491361,"[tiger shark, Galeocerdo cuvieri]"
4,n01494475,"[hammerhead, hammerhead shark]"


On vient de récupérer les identifiants et les noms des animaux, ou d'objet. Nous allons maintenant récupérer les identifiants des animaux dans le fichier `animals.csv` et les sauvegarder dans un nouveau fichier `animals_id.csv`. Commençons par lire le fichier csv.

In [10]:
# Lecture du fichier CSV
df_animals = pd.read_csv('animals.csv')
df_animals.head()

,animal,animalLabel
0,http://www.wikidata.org/entity/Q140,lion
1,http://www.wikidata.org/entity/Q5113,birds
2,http://www.wikidata.org/entity/Q14334,wolverine
3,http://www.wikidata.org/entity/Q14272,Geraniales
4,http://www.wikidata.org/entity/Q19064,Agyrtidae


Comparons-les animalLabel avec les noms des animaux dans le fichier `LOC_synset_mapping.txt`. Pour cela, nous allons créer une fonction qui va comparer les deux listes et retourner l'identifiant de l'animal si il y a une correspondance.

In [8]:
# Définissons la liste des identifiants des animaux
animals_id = []

# Parcourons le CSV récupéré de WikiData avec df['name'], si le nom de l'animal : df_animals['animalLabel'], est dans le tableau df['name'], on récupère l'identifiant de l'animal et on l'ajoute dans la liste animals_id
for animal in df_animals['animalLabel']:
    for i in range(len(df['name'])):
        if animal in df['name'][i]:
            animals_id.append(df['id'][i])
            break

animal


TypeError: string indices must be integers

In [30]:
# En mieux

# Définissons la liste des identifiants et des url Wikidata des animaux

ok = {"id": [], "name": []}

for id, name in zip(df['id'], df.name):
        for n in name:
            ok["id"].append(id)
            ok["name"].append(n)

okdf = pd.DataFrame(ok)
temps = pd.merge(df_animals, okdf, left_on='animalLabel', right_on='name')



temps = temps.dropna()

#animals_id = temps['id'].unique()

animals = temps.groupby(['animal', 'id']).first().reset_index()
animals = animals.drop('name', axis=1)  # Supprimer la colonne 'name'
animals.to_csv('animals_id.csv', index=False)

                                  animal animalLabel         id      name
0    http://www.wikidata.org/entity/Q140        lion  n02129165      lion
1  http://www.wikidata.org/entity/Q19125    scorpion  n01770393  scorpion
2  http://www.wikidata.org/entity/Q15343    platypus  n01873310  platypus
3  http://www.wikidata.org/entity/Q19939       tiger  n02129604     tiger
4   http://www.wikidata.org/entity/Q8332     red fox  n02119022   red fox


In [25]:
# Enregistrons les identifiants dans un fichier CSV
df_animals_id = pd.DataFrame({'id': animals_id})
df_animals_id.to_csv('animals_id.csv', index=False)
df_animals_id.head()

ValueError: Per-column arrays must each be 1-dimensional